<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/SecantNewtonBroyden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
from jax.config import config
from scipy.optimize import minimize
config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots

In [ ]:
def f(x):
    return 0.5*(x-0.8)*(x-0.3)*(x+0.5)

In [ ]:
xplot = np.linspace(-0.7,1.,100)
fig=make_subplots()
fig.add_scatter(x=xplot, y=f(xplot), mode='lines')
fig.update_layout(width=600,height=600,template='plotly_dark')

In [ ]:
def secant(f, x0, x1, maxiter=50, tol=1e-12):
    f0=f(x0)
    f1=f(x1)
    xylist=[[x0,f0], [x1,f1]]
    for i in range(maxiter):
        x2 = (x1-x0)/(f0-f1)*f1 +x1
        f2 = f(x2)
        xylist.append([x2,f2])
        if abs(f2)<tol:
            break
        x0, f0 = x1, f1
        x1, f1 = x2, f2
    return x2, f2, np.array(xylist)

In [ ]:
secant(f, 0., 0.5)

(0.3000000000000018,
 -3.6637359812630124e-16,
 array([[ 0.00000000e+00,  6.00000000e-02],
        [ 5.00000000e-01, -3.00000000e-02],
        [ 3.33333333e-01, -6.48148148e-03],
        [ 2.87401575e-01,  2.54249327e-03],
        [ 3.00342789e-01, -6.85400857e-05],
        [ 3.00003080e-01, -6.15901299e-07],
        [ 2.99999999e-01,  1.58567082e-10],
        [ 3.00000000e-01, -3.66373598e-16]]))

In [ ]:
def newton(f, x0, maxiter=50, tol=1e-12):
    fprime = jax.grad(f)
    f0=f(x0)
    xylist = [[x0,f0]]
    for i in range(maxiter):
        x1 = x0 - f0/fprime(x0)
        f1 = f(x1)
        xylist.append([x1,f1])
        if abs(f1)<tol:
            break
        x0,f0 =x1,f1
    return x1, f1, jnp.array(xylist)

In [ ]:
newton(f,0.)

(DeviceArray(0.3, dtype=float64, weak_type=True),
 DeviceArray(-0., dtype=float64, weak_type=True),
 DeviceArray([[ 0.00000000e+00,  6.00000000e-02],
              [ 3.87096774e-01, -1.59511262e-02],
              [ 2.88931283e-01,  2.23144280e-03],
              [ 2.99916207e-01,  1.67595823e-05],
              [ 2.99999995e-01,  1.05246061e-09],
              [ 3.00000000e-01, -0.00000000e+00]], dtype=float64))

In [ ]:
def fun(x):
    return jnp.array([x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]])

In [ ]:
def newtonN(f,x0,maxiter=50, tol=1e-12):
    f_jac=jax.jacobian(f)
    f0 = f(x0)
    for i in range(maxiter):
        x1 = x0 - jnp.linalg.inv(f_jac(x0)) @ f0
        f1 = f(x1)
        if jnp.linalg.norm(f1)<tol:
            break
        x0, f0 = x1, f1
    return x1, f1



In [ ]:
newtonN(fun, jnp.array([0.,0.]))

(DeviceArray([0.8411639, 0.1588361], dtype=float64),
 DeviceArray([ 0.00000000e+00, -5.55111512e-17], dtype=float64))

In [ ]:

def broyden(f, x0, jac, maxiter=5, tol=1e-10):
    Jinv=np.linalg.inv(jac(x0))
    f0 = f(x0)
    for i in range(maxiter):
        dx = - Jinv @ f0
        x1 = x0 + dx
        f1 = f(x1)
        alpha = 1.
        jac_calc=False
        while (np.linalg.norm(f1) > np.linalg.norm(f0)):
            alpha = alpha * 0.75
            if alpha < 1e-7:
                jac_calc=True
                Jinv=np.linalg.inv(jac(x0))
                dx = - Jinv @ f0
            x1 = x0 + alpha*dx
            f1 = f(x1)
        
        print(jac_calc)
        if np.linalg.norm(f1)<tol:
            break
        dx, df = x1 - x0, f1-f0
        den = (dx.T @ Jinv @ df)
        # print(den, np.linalg.norm(f1))
        Jinv = Jinv + ((dx - Jinv@df) @ (dx.T @ Jinv)) / den
        x0, f0 = x1, f1
    return x1, f1

In [ ]:
x0 = jnp.array([0.,0.])

In [ ]:
broyden(fun, x0, jax.jacobian(fun), maxiter=50)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False


(DeviceArray([0.8411639, 0.1588361], dtype=float64),
 DeviceArray([-6.67466082e-13,  6.67382816e-13], dtype=float64))

##Demonstrate Broyden method yields minimum Frobenius norm subject to secant equation

In [ ]:
N=5
rng=np.random.RandomState(123)
Jold=rng.uniform(size=(N,N))
dx = rng.uniform(size=(N,1))
df = rng.uniform(size=(N,1))

In [ ]:
Jold

array([[0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897],
       [0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752],
       [0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426],
       [0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759],
       [0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338]])

In [ ]:
def frobenius(x):
    Jnew=x.reshape((N,N))
    return np.sum((Jnew - Jold)**2)

In [ ]:
def secant_constraint(x):
    Jnew=x.reshape((N,N))
    return 1e-6 - np.linalg.norm(Jnew @ dx - df)

In [ ]:
res=minimize(frobenius, Jold.flatten(), method='SLSQP', constraints=[{'type':'ineq', 'fun':secant_constraint}])
res

     fun: 2.82801690489399
     jac: array([-0.756679  , -0.84765553, -0.53481111, -0.68815953, -1.47835046,
       -0.50452325, -0.56518278, -0.35659063, -0.45883724, -0.98570487,
       -0.2615872 , -0.29303825, -0.18488663, -0.23789984, -0.51107234,
       -0.28661236, -0.32107216, -0.2025739 , -0.2606588 , -0.55996454,
       -0.74246693, -0.83173466, -0.52476618, -0.67523441, -1.45058411])
 message: 'Optimization terminated successfully'
    nfev: 88
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([ 0.31812983, -0.13768827, -0.04055399,  0.20723514, -0.01970598,
        0.17084479,  0.69817276,  0.50653438,  0.25151324, -0.10073501,
        0.21238437,  0.58253053,  0.34612889, -0.05927206,  0.142508  ,
        0.59468914,  0.02195555,  0.07416474,  0.40122191,  0.25184516,
        0.26316751,  0.43356448,  0.46207225,  0.27340632, -0.00284864])

In [ ]:
secant_constraint(res.x)

-1.3217346392056341e-07

In [ ]:
frobenius(res.x)

2.82801690489399

In [ ]:
Jold + ((df - Jold@dx) @ dx.T)/(jnp.linalg.norm(dx))**2

DeviceArray([[ 0.31812947, -0.13768862, -0.04055422,  0.20723482,
              -0.01970656],
             [ 0.17084458,  0.69817257,  0.50653427,  0.25151307,
              -0.10073534],
             [ 0.21238423,  0.58253042,  0.34612887, -0.05927214,
               0.14250785],
             [ 0.59468906,  0.02195549,  0.07416466,  0.40122183,
               0.25184497],
             [ 0.26316716,  0.4335641 ,  0.46207202,  0.27340601,
              -0.00284908]], dtype=float64)

In [ ]:
(res.x).reshape(N,N)

array([[ 0.31812983, -0.13768827, -0.04055399,  0.20723514, -0.01970598],
       [ 0.17084479,  0.69817276,  0.50653438,  0.25151324, -0.10073501],
       [ 0.21238437,  0.58253053,  0.34612889, -0.05927206,  0.142508  ],
       [ 0.59468914,  0.02195555,  0.07416474,  0.40122191,  0.25184516],
       [ 0.26316751,  0.43356448,  0.46207225,  0.27340632, -0.00284864]])